# Import Yellow Taxi Data - Part 2

Create a Hive external table using the S3 Parquet files.


## Create Hive Table

In [17]:
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [18]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [19]:
%config SqlMagic.named_parameters = "enabled"

In [20]:
%sql trino://admin@${DOCKER_HOST_OR_IP}:8443/hive --connection_arguments '{"http_scheme":"https", "verify": false}'

In [21]:
%%sql 

show catalogs

Running query in 'trino://admin@10.143.11.241:8443/hive'

Catalog
hive
iceberg
system
tpcds
vast


In [22]:
import os
S3A_BUCKET = os.getenv('S3A_BUCKET')

In [23]:
! echo {S3A_BUCKET}

csnow-bucket


In [24]:
%%sql

CREATE SCHEMA IF NOT EXISTS hive.nyt

Running query in 'trino://admin@10.143.11.241:8443/hive'

++
||
++
++

In [25]:
%%sql

SHOW SCHEMAS

Running query in 'trino://admin@10.143.11.241:8443/hive'

Schema
default
information_schema
nyt
social_media


In [38]:
%%sql

DROP TABLE IF EXISTS nyt.yellow_trip_data

Running query in 'trino://admin@10.143.11.241:8443/hive'

++
||
++
++

In [39]:
%%sql

CREATE TABLE nyt.yellow_trip_data (
  vendorid VARCHAR,
  tpep_pickup_datetime VARCHAR,
  tpep_dropoff_datetime VARCHAR,
  passenger_count BIGINT,
  trip_distance DOUBLE,
  ratecodeid BIGINT,
  store_and_fwd_flag DOUBLE,
  pulocationid INT,
  dolocationid INT,
  payment_type VARCHAR,
  fare_amount DOUBLE,
  extra DOUBLE,
  mta_tax DOUBLE,
  tip_amount DOUBLE,
  tolls_amount DOUBLE,
  improvement_surcharge DOUBLE,
  total_amount DOUBLE,
  congestion_surcharge DOUBLE,  
  airport_fee DOUBLE,
  dropoff_latitude DOUBLE,
  dropoff_longitude DOUBLE,
  pickup_latitude DOUBLE,
  pickup_longitude DOUBLE,
  surcharge DOUBLE
)
WITH (
  format = 'PARQUET',
  external_location = 's3a://{{S3A_BUCKET}}/yellow_tripdata/'
)

Running query in 'trino://admin@10.143.11.241:8443/hive'

++
||
++
++

In [43]:
%%sql
    
SELECT format('%,d', COUNT(*)) as row_count FROM nyt.yellow_trip_data

Running query in 'trino://admin@10.143.11.241:8443/hive'

row_count
"1,763,456,499"


In [41]:
%%sql

SELECT *
FROM nyt.yellow_trip_data
LIMIT 100

Running query in 'trino://admin@10.143.11.241:8443/hive'

passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,dropoff_latitude,dropoff_longitude,pickup_latitude,pickup_longitude,surcharge
2,9.8,None,0.0,-999,-999,24.1,-999.0,None,5.65,4.15,-999.0,33.9,-999.0,-999.0,40.774278,-73.87347099999998,40.762107,-73.978843,0.0
1,2.2,None,0.0,-999,-999,9.3,-999.0,None,1.54,0.0,-999.0,10.84,-999.0,-999.0,40.747773,-73.97288899999998,40.719872,-73.987342,0.0
1,3.7,None,0.0,-999,-999,14.5,-999.0,None,2.32,0.0,-999.0,16.82,-999.0,-999.0,40.74506,-73.997792,40.77255,-73.95920099999998,0.0
3,1.7,None,0.0,-999,-999,10.1,-999.0,None,0.0,0.0,-999.0,10.1,-999.0,-999.0,40.759942,-73.984448,40.738061,-73.99652299999998,0.0
1,1.6,None,0.0,-999,-999,8.9,-999.0,None,0.0,0.0,-999.0,8.9,-999.0,-999.0,40.752788,-73.98509199999998,40.760026,-73.96490199999998,0.0
1,0.6999999999999998,None,0.0,-999,-999,5.3,-999.0,None,0.0,0.0,-999.0,5.3,-999.0,-999.0,40.719782,-74.010428,40.722629,-74.003809,0.0
3,3.1,None,0.0,-999,-999,12.9,-999.0,None,0.0,0.0,-999.0,12.9,-999.0,-999.0,40.751457,-74.001508,40.763873,-73.96203,0.0
1,1.45,None,None,-999,-999,6.9,-999.0,None,0.0,0.0,-999.0,6.9,-999.0,-999.0,40.750332,-73.99134499999998,40.768275,-73.982107,0.0
2,0.76,None,None,-999,-999,5.7,-999.0,None,4.0,0.0,-999.0,9.7,-999.0,-999.0,40.760553,-73.97216,40.752022,-73.97732999999998,0.0
1,0.54,None,None,-999,-999,4.9,-999.0,None,0.0,0.0,-999.0,5.4,-999.0,-999.0,0.0,0.0,0.0,0.0,0.5


In [30]:
%%sql

SHOW CREATE TABLE nyt.yellow_trip_data

Running query in 'trino://admin@10.143.11.241:8443/hive'

Create Table
"CREATE TABLE hive.nyt.yellow_trip_data ( vendorid varchar, tpep_pickup_datetime varchar, tpep_dropoff_datetime varchar, passenger_count bigint, trip_distance double, ratecodeid bigint, store_and_fwd_flag double, pulocationid integer, dolocationid integer, payment_type varchar, fare_amount double, extra double, mta_tax double, tip_amount double, tolls_amount double, improvement_surcharge double, total_amount double, congestion_surcharge double, airport_fee double, dropoff_latitude double, dropoff_longitude double, pickup_latitude double, pickup_longitude double, surcharge double)WITH ( external_location = 's3a://csnow-bucket/yellow_tripdata', format = 'PARQUET')"


In [31]:
%%sql

EXPLAIN
    SELECT *
    FROM nyt.yellow_trip_data
    LIMIT 100


Running query in 'trino://admin@10.143.11.241:8443/hive'

Query Plan
